In [1]:
import sys
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

sys.path.append('../')

from utils.io import recursive_scan2df
from utils import endoscopy

In [2]:
folder = 'img/21_04_02_calib'
df = recursive_scan2df('{}'.format(folder), '.npy')
df

# bilateral filter on rgb
# doesnt matter if hsv or rgb -> hsv more intuitive though
# radius: gradient over search
# average area -> if mean > th, valid circle found, else previous or uninitialized

crp_count = 0
for _, row in df.iterrows():
    img = np.load(os.path.join(folder, row.folder, row.file))
    # mask_bil_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    bil_img = cv2.bilateralFilter(img, 15, 75, 75) # bilateral on rgb
    # bil_img = cv2.fastNlMeansDenoisingColored(img, 10, 10)
    bil_hsv = cv2.cvtColor(bil_img, cv2.COLOR_BGR2HSV)

    # mask
    th = 20
    mask_bil_hsv = cv2.bitwise_not(cv2.inRange(bil_hsv, (0, 0, 0), (255, 255, th)))
    # mask_bil_hsv = cv2.bitwise_not(cv2.inRange(bil_img, (0, 0, 0), (th, th, th)))

    # circle
    hsv_center, hsv_radius = endoscopy.boundaryCircle(mask_bil_hsv)
    top_left, shape, col_mean, row_mean = endoscopy.boundaryRectangle(mask_bil_hsv)

    # confidence measure for elumination
    # where(meshgrid - center < radius).mean
    level = endoscopy.illuminationLevel(mask_bil_hsv, hsv_center, hsv_radius)

    # crop
    if level > 0.98:
        inner_top_left, inner_shape = endoscopy.maxRectangleInCircle(img.shape, hsv_center, hsv_radius)
        inner_top_left, inner_shape = inner_top_left.astype(np.int), tuple(map(int, inner_shape))
        crp = endoscopy.crop(img, inner_top_left, inner_shape)
        crp_count += 1
        np.save('img/crop_{}.npy'.format(crp_count), crp)


    # int
    hsv_center, hsv_radius = hsv_center.astype(np.int), np.int(hsv_radius)
    top_left, shape = top_left.astype(np.int), tuple(map(int, shape))

    # bgr
    mask_bil_hsv = cv2.cvtColor(mask_bil_hsv, cv2.COLOR_GRAY2BGR)

    # draw
    cv2.circle(img, (hsv_center[1], hsv_center[0]), hsv_radius, (255, 255, 0), 2)
    cv2.circle(img, (hsv_center[1], hsv_center[0]), 2, (255, 255, 0), -1)

    cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1] + shape[1], top_left[0] + shape[0]), (255, 0, 255), 2)

    cv2.imshow('img', img)
    cv2.imshow('mask_bil_hsv', mask_bil_hsv)
    cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
df = recursive_scan2df('.', '.npy')

for _, row in df.iterrows():
    img = np.load(os.path.join(row.folder, row.file))
    cv2.imwrite(os.path.join(row.folder, '{}.png'.format(row.file.split('.')[0])), img)


In [5]:
df = recursive_scan2df('.', '.npy')
df

,folder,file
0,img,img_142.npy
1,img,img_54.npy
2,img,img_90.npy
3,img,img_161.npy
4,img,img_85.npy
...,...,...
187,img,img_30.npy
188,img,img_152.npy
189,img,img_80.npy
190,img,img_9.npy


In [2]:
import sys
import cv2
import os
import numpy as np

sys.path.append('../')

from utils.io import scan2df

# homography estimation
folder = 'img'
crp_df = scan2df(folder, '.npy')

for idx in range(len(crp_df) - 1):

    crp_0 = np.load(os.path.join(crp_df.iloc[idx].folder, crp_df.iloc[idx].file))
    crp_1 = np.load(os.path.join(crp_df.iloc[idx+1].folder, crp_df.iloc[idx+1].file))

    # homography estimation
    patternSize = (4, 11)

    # convert to grayscale
    crp_0 = cv2.cvtColor(crp_0.astype(np.float32), cv2.COLOR_BGR2GRAY).astype(np.uint8)
    crp_1 = cv2.cvtColor(crp_1.astype(np.float32), cv2.COLOR_BGR2GRAY).astype(np.uint8)

    # find points
    found0, pts0 = cv2.findCirclesGrid(crp_0, patternSize=patternSize, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)
    found1, pts1 = cv2.findCirclesGrid(crp_1, patternSize=patternSize, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)

    if found0 and found1:
        G, _ = cv2.findHomography(pts0, pts1, cv2.RANSAC)

        wrp = cv2.warpPerspective(crp_0, G, crp_1.shape[-2:][::-1])

        cv2.imshow('crp_0', crp_0)
        cv2.imshow('crp_1', crp_1)
        cv2.imshow('wrp', wrp)
        cv2.imshow('error', np.abs(wrp - crp_1))
        cv2.waitKey()
    else:
        print('Did not find points.')


    cv2.destroyAllWindows()

Did not find points.
Did not find points.
Did not find points.
Did not find points.
